In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import fuzzywuzzy
from fuzzywuzzy import fuzz
from itertools import combinations
from joblib import Parallel, delayed

/Users/yihuiwang/Envs/ivy/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import multiprocessing
print(multiprocessing.cpu_count())

4


test with 100 abstract + title w/ all theory list

In [3]:
fname = 'pub_abstract.csv'

In [4]:
dfx = pd.read_csv(fname)

In [5]:
len(dfx)

61524

---

In [60]:
df = pd.read_csv(fname, nrows=1000)
df.head(2)

,abstract,title,scopus_id
0,"Within the field of information systems (IS), ...",Rethinking the concept of user involvement,24044550516
1,The Business Process Execution Language for We...,From business process models to web services o...,58049112405


In [61]:
df_clean = df[['abstract','title']]
df_clean['combine'] = df['abstract'] +' ' + df['title']

In [62]:
data = df_clean['combine'].to_frame()
data['ratio'] = 0
data['theory'] = 'theory'

In [9]:
theo_list = pd.read_csv('theory_list.csv')

---

In [10]:
from itertools import product 

In [63]:
combine_list = data['combine'].tolist()
theory_list = theo_list['Theory'].tolist()

In [64]:
list_combine_theory = list(product(combine_list,theory_list))

In [66]:
len(list_combine_theory)

107000

In [67]:
list2 = np.array_split(list_combine_theory, 1000)

In [68]:
list2[1]

array([['The Business Process Execution Language for Web Services (BPEL) has emerged as the de facto standard for implementing business processes. At the same time, Model Driven Architecture (MDA) is being applied to the field of business process engineering by separating business logic from the underlying platform technology. However, due to the challenge of mapping graph-oriented modeling languages to block-structured ones and the informal description of UML 2.0 Activity Diagram (AD) and BPEL, transforming AD models to executable BPEL code is not trivial. This paper proposes an approach to transform AD to BPEL and paves the way for further general transformation between graph-oriented and block-structured process modeling languages. © 2008 Springer Berlin Heidelberg. From business process models to web services orchestration: The case of UML 2.0 activity diagram to BPEL',
        'Absorptive capacity theory'],
       ['The Business Process Execution Language for Web Services (BPEL) h

In [69]:
def fun(x):
    aff_pair_score = pd.DataFrame(columns=('Affiliation1','Affiliation2','Token_Set_Ratio'))
    for entry in x:
        str1 = entry[0]
        str2 = entry[1]
        Token_Set_Ratio = fuzz.token_set_ratio(str1.tolower(), str2.tolower())
        aff_pair_score = aff_pair_score.append(pd.DataFrame({'Affiliation1':[str1],'Affiliation2':[str2],'Token_Set_Ratio':[Token_Set_Ratio]}),ignore_index=True)
    return aff_pair_score

In [71]:
%%time
y = Parallel(n_jobs= 3)(delayed(fun)(list2[i])for i in range(1000))

CPU times: user 12.5 s, sys: 10.1 s, total: 22.7 s
Wall time: 4min 22s


In [72]:
aff_select = pd.DataFrame(columns=('Affiliation1','Affiliation2','Token_Set_Ratio'))
for index in range(1000):
    df_in = y[index]
    aff_a = df_in.loc[(df_in['Token_Set_Ratio']>70)]
    aff_select = aff_select.append(aff_a)

In [ ]:
aff_select.reset_index(drop=True)  # reset index

In [73]:
aff_select

,Affiliation1,Affiliation2,Token_Set_Ratio
45,"Within the field of information systems (IS), ...",Information warfare,73
89,The Business Process Execution Language for We...,Structured process modeling theory (SPMT),82
93,The Business Process Execution Language for We...,Technology acceptance model,74
45,Logistics management is defined as that part o...,Information warfare,73
26,© 2014 IEEE.Multimedia content in websites pos...,Dynamic capabilities,75
18,The status quo of prohibiting broadband servic...,Critical social theory,81
45,Recent advances in electronics and telecommuni...,Information warfare,73
85,Recent advances in electronics and telecommuni...,Socio-technical theory,81
15,The complexity of evaluating expert system (ES...,Complexity theory,74
45,A prototype Project Management Quality Cost Sy...,Information warfare,73
